In [1]:
import ee
ee.Initialize()

import geemap
ee.Initialize()

# Define a simple AOI (Los Lagos region)
aoi = ee.Geometry.Polygon(
    [[[-73.11, -41.07],
      [-72.94, -41.07],
      [-72.94, -41.17],
      [-73.11, -41.17]]])

# Load a Sentinel-2 image
image = (
    ee.ImageCollection('COPERNICUS/S2_SR')
    .filterBounds(aoi)
    .filterDate('2023-01-01', '2023-12-31')
    .sort('CLOUDY_PIXEL_PERCENTAGE')
    .first()
)

# Visualize
Map = geemap.Map()
Map.centerObject(aoi, 8)
Map.addLayer(image, {'min': 0, 'max': 3000, 'bands': ['B4', 'B3', 'B2']}, 'Sentinel-2')
Map

/opt/miniconda3/envs/geoai/lib/python3.11/site-packages/ee/deprecation.py:209: DeprecationWarning: 

Attention required for COPERNICUS/S2_SR! You are using a deprecated asset.
To make sure your code keeps working, please update it.
Learn more: https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR

  warnings.warn(warning, category=DeprecationWarning)


Map(center=[-41.120018539438874, -73.02500000000035], controls=(WidgetControl(options=['position', 'transparen…

In [2]:
import json

# Create a map centered on your region
Map = geemap.Map(center=[-41.3, -72.8], zoom=9)

# Use the drawing tools to create a polygon
# Click the polygon tool and draw your AOI
Map

Map(center=[-41.3, -72.8], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright'…

In [5]:
# Get the last drawn feature
aoi = Map.draw_last_feature
# Convert to GeoJSON
aoi_geojson = geemap.ee_to_geojson(aoi)
# Save to file
with open('../data/external/aoi.geojson', 'w') as f:
    json.dump(aoi_geojson, f, indent=2)
print("✓ AOI saved to data/external/aoi.geojson")

✓ AOI saved to data/external/aoi.geojson


In [8]:
import json, ee, geemap

# Load AOI
with open('../data/external/aoi.geojson', 'r') as f:
    aoi_geojson = json.load(f)
aoi_ee = geemap.geojson_to_ee(aoi_geojson)

# Map
Map = geemap.Map()
Map.add_basemap('SATELLITE')  # add basemap first

# Draw an outline regardless of geometry/feature/collection
outline = ee.Image().byte().paint(aoi_ee, 1, 3)  # value=1, width=3 px
Map.addLayer(outline, {'palette': 'red'}, 'AOI outline')

# Center on AOI
Map.centerObject(aoi_ee, 8)
Map

Map(center=[-41.12099664217866, -73.056936000002], controls=(WidgetControl(options=['position', 'transparent_b…